- join tables
- remove duplicates
- outliers / missing data
- standarize names -> coordinates to names of places
- ...

deixo aixo aqui per si pot servir despres

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb
sys.path.append('..')
from helper import *
setwd()

Objects = Objects()

In [2]:
import geopy.geocoders
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'app_adsdb'

#geolocator = Nominatim(user_agent='app_adsdb/1')
geolocator = Nominatim(timeout=3)

In [3]:

def postcode(df,latitude="Latitude",longitude="Longitude"):
    Lat = np.array(df[[latitude]]).flatten()
    Long = np.array(df[[longitude]]).flatten()

    coordinates = [(Lat[i],Long[i]) for i in range(len(Lat))]#len(Lat)
    location = [geolocator.reverse(coor).raw["address"] for coor in coordinates]

    postcode = []

    for loc in location:
        if "postcode" in loc:
            postcode.append(loc["postcode"])
        else:
            postcode.append("")

    return df.assign(ZIP=list(postcode))
    


In [7]:

con = duckdb.connect(database="data/formatted/db_housing.db", read_only=True)
df = con.execute("SELECT * FROM test_housing_v1_20221008_161949").fetchdf()

df2 = df.head(10)
df3 = postcode(df2,latitude="lat",longitude="long")
df3.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state,ZIP
0,7049044568,https://reno.craigslist.org/apa/d/reno-beautif...,reno / tahoe,https://reno.craigslist.org,1148,apartment,1078,3,2.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/01616_daghmBUvTC...,Ridgeview by Vintage is where you will find al...,39.5483,-119.796,ca,89512
1,7043634882,https://reno.craigslist.org/apa/d/sparks-state...,reno / tahoe,https://reno.craigslist.org,1813,apartment,1683,2,2.0,1,...,0,0,w/d in unit,attached garage,https://images.craigslist.org/00t0t_erYqC6LgB8...,2BD | 2BA | 1683SQFTDiscover exceptional servi...,39.6269,-119.708,ca,89441
2,7049043759,https://reno.craigslist.org/apa/d/reno-no-long...,reno / tahoe,https://reno.craigslist.org,289,apartment,250,0,1.0,1,...,0,1,laundry on site,NaN,https://images.craigslist.org/01616_fALAWFV8zQ...,"Move In Today: Reno Low-Cost, Clean & Furnishe...",39.5357,-119.805,ca,89512
3,7049041899,https://reno.craigslist.org/apa/d/reno-1x1-fir...,reno / tahoe,https://reno.craigslist.org,1095,apartment,708,1,1.0,1,...,0,0,w/d in unit,carport,https://images.craigslist.org/00303_3HSJz75zlI...,MOVE IN SPECIAL FREE WASHER/DRYER WITH 6 OR 12...,39.4477,-119.771,ca,89511
4,7049041434,https://reno.craigslist.org/apa/d/reno-6850-sh...,reno / tahoe,https://reno.craigslist.org,1295,condo,930,2,2.0,0,...,0,0,w/d in unit,carport,https://images.craigslist.org/01313_8AoC9NJZWA...,6850 Sharlands Ave E-1021 Reno NV 89523 ...,39.5193,-119.897,ca,89523


In [4]:
def formatted2trusted():
    
    # list of tables already processed:
    processed_table = []
    with open('logs/processed_data_trusted.txt', "r") as f0:
        for line in f0:
            processed_table.append(line.rstrip())
    
    # process datasets:
    for datasource in Objects:
        id = datasource["id"]

        # conect to DB
        con = duckdb.connect(database="data/formatted/db_{}.db".format(id), read_only=True)

        # take all tables names
        tables = con.execute("SHOW TABLES").fetchall()
        tables_names = [x[0] for x in tables]

        # tables to process:
        tables_not_processed = [table for table in tables_names if not table in processed_table]
        if len(tables_not_processed) == 0:
            continue
        
        # list of all tables in dataframe format
        dataframes = []
        for table in tables_not_processed:
            dataframes.append(con.execute("SELECT * FROM {}".format(table)).fetchdf())

        # close database
        con.close()

        # merge all dataframes
        df = pd.concat(dataframes)#,verify_integrity=True

        #column_names = list(df.columns.values)
        #if "ZIP" not in column_names:
            #print("No ZIP",len(df))
            #df2 = df.head(100)
            #postcode(df2,latitude="lat",longitude="long")

        con = duckdb.connect(database="data/trusted/db_{}.db".format(id), read_only=False)
        print("CREATE TABLE {} AS SELECT * FROM df".format(id))
        con.execute("CREATE TABLE {} AS SELECT * FROM df".format(id))
        
        

        with open('logs/processed_data_trusted.txt', "a") as f0:
            for table in tables_not_processed:
                f0.write(table + "\n")

    

In [6]:
formatted2trusted()

CREATE TABLE housing AS SELECT * FROM df
CREATE TABLE schools AS SELECT * FROM df
CREATE TABLE hospitals AS SELECT * FROM df
